**EE20MTECH12001 : Shantanu Yadav**

**Problem Statement**

**Problem 1**

We are tasked with designing a box shaped structure with width w, height h, and depth d. We are given
that the total wall area is at most 200 square units, the total floor area is at most 60 square units, and
the aspect ratios (h/w and d/w) are at least 0.8 and at most 1.2. Formulate an optimization program to
solve for the dimensions h, w and d that results in a box of the largest possible volume, and implement
in CVX/CVXPY.

In [ ]:
import numpy as np
import cvxpy as cp
y1=cp.Variable(1,name='y1') 
y2=cp.Variable(1,name='y2') 
y3=cp.Variable(1,name='y3') 
h=cp.exp(y1)
w=cp.exp(y2)
d=cp.exp(y3)
#objective
objective=cp.Minimize(-(y1+y2+y3))
#constraints
constraints = [2*(cp.exp(y1+y2)+cp.exp(y1+y3))<=200, cp.exp(y2+y3)<=60, cp.exp(y1-y2)<=1.2, cp.exp(y3-y2)<=1.2, 0.8*cp.exp(y2-y3)<=1, 0.8*cp.exp(y2-y1)<=1]
#solve problem
prob = cp.Problem(objective, constraints);
print(prob)
prob.solve()
print('\n')
print ('y1 = ',y1.value[0])
print ('h =', np.exp(y1.value[0]))
print ('y2 =',y2.value[0])
print ('w =', np.exp(y2.value[0]))
print ('y3 =',y3.value[0])
print ('d =', np.exp(y3.value[0]))
print ('optimal volume is ', np.exp(y1.value[0])*np.exp(y2.value[0])*np.exp(y3.value[0]))

minimize -y1 + y2 + y3
subject to 2.0 * (exp(y1 + y2) + exp(y1 + y3)) <= 200.0
           exp(y2 + y3) <= 60.0
           exp(y1 + -y2) <= 1.2
           exp(y3 + -y2) <= 1.2
           0.8 * exp(y2 + -y3) <= 1.0
           0.8 * exp(y2 + -y1) <= 1.0


y1 =  1.864850713424119
h = 6.454972173365166
y2 = 2.047173283710832
w = 7.745974458523242
y3 = 2.0471711521402454
d = 7.745957947449518
optimal volume is  387.2982814586268


**Problem 2**

In [ ]:
import numpy as np
import cvxpy as cvx
def min_SINR(G,P_max,P_received,sigma,Group,Group_max,epsilon = 0.05):

  n,m = np.shape(G)

  delta = np.identity(n)
  SignalMatrix = G*np.eye(n) 
  InterferenceMatrix = G-SignalMatrix 
  num_groups = int(np.size(Group,0))
  Group_norm = np.array([group/np.sum(group) for group in Group])
  p = cvx.Variable((n,1))
  optimalPow = np.zeros(n)
  # upper and lower bounds
  u = 1e4
  l = 0
  
  alpha = cvx.Parameter()
  constraints = [InterferenceMatrix@p + sigma <= alpha*SignalMatrix*p, p <= P_max, p >= 0, G*p <= P_received, Group_norm*p <= Group_max]

  obj = cvx.Minimize(alpha)
  alpha.value = u
  prob = cvx.Problem(obj, constraints)
  prob.solve()
  if prob.status != 'optimal':
    print('No optimal solution within bounds\n')
    return 'Error: no optimal solution within bounds',np.nan,np.nan,np.nan

  alpha.value = l
  prob = cvx.Problem(obj, constraints)
  prob.solve()
  if prob.status == 'optimal':
    print('No optimal solution within bounds\n')
    return 'Error: no optimal solution within bounds',np.nan,np.nan,np.nan

  # Bisection algortithm starts
  maxLoop = int(1e7)
  for i in range(1,maxLoop):
    alpha.value = (u + l)/2.0
    
    if u-l <= epsilon:
      break
    prob = cvx.Problem(obj, constraints)
    prob.solve()
    if prob.status == 'optimal':
      u = alpha.value
      optimalPow = p.value
    else:
      l = alpha.value
    if u - l > epsilon and i == (maxLoop-1):
      print("Solution not converged to order epsilon")
  return l,u,alpha.value,optimalPow


In [ ]:
  G = np.array([[1.0,0.1,0.2,0.1,0.0],
                [0.1,1.0,0.1,0.1,0.0],
                [0.2,0.1,2.0,0.2,0.2],
                [0.1,0.1,0.2,1.0,0.1],
                [0.0,0.0,0.2,0.1,1.0]])

  n,m = np.shape(G)
  P_max = np.array([3.]*n).reshape((n,1))
  P_received = (np.array([5.,5.,5.,5.,5.]).reshape((n,1)))/n
  sigma = np.array([0.5,0.5,0.5,0.5,0.5]).reshape((n,1))
  Group = np.array([[1.,1.,0,0,0],[0,0,1.,1.,1.]])
  Group_max = np.array([[3.0],[6.0]])
  l,u,alpha,best=min_SINR(G,P_max,P_received,sigma,Group,Group_max)
  print('Minimum SINR=%.4g'%(1/alpha))
  print('Power=%s'%(best))

Minimum SINR=0.7598
Power=[[0.51316491]
 [0.49975582]
 [0.32181663]
 [0.54193416]
 [0.46441846]]
